<a href="https://colab.research.google.com/github/ntb-cicles/SMB-Linux-Windows/blob/main/Samba_com_a_controlador_de_domini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grup de treball VS Domini

Un **WORKGROUP** (grup de treball) és un **conjunt de màquines connectades a una mateixa xarxa**. En un workgroup **cada màquina gestiona de forma local els seus usuaris i contrasenyes**.

\

Un **DOMAIN** (domini) és també un **conjunt de màquines connectades a una mateixa xarxa**. Però en un domain, és **només una màquina, amb rol de servidor, qui gestiona els usuaris i contrasenyes de totes les màquines del domini de la xarxa**. 

\

Usualment els dominis s'utilitzen en xarxes amb un gran nombre d'equips, per exemple, en una empresa on hi han 60 ordinadors, és molt més fàcil gestionar tots els usuaris i contrasenyes dels 60 ordinadors des d'un servidor que pas haver d'anar un per un de forma local.

# Samba com a controlador de domini

Feu una nova còpia de seguretat del vostre fitxer de configuració de SAMBA (smb.conf) ja que tornarem a canviar-lo.

Comanda "hostnamectl" per saber el nom de la màquina que utilitzarem com a servidor.

In [ ]:
hostnamectl

In [ ]:
# Exemple de resultat de la comanda "hostnamectl"
nil@servnil:/etc$ hostnamectl

        Static hostname: servnil
        Icon name: computer-vm
        Chassis: vm
        .....

In [ ]:
timedatectl

El ***hostname*** (en català, ***nom d'equip***) és un nom únic i relativament informal que es dóna a un dispositiu connectat a una xarxa

Si no teniu el directori "/etc/samba" cal que instal·leu samba amb la comanda següent:

In [ ]:
sudo apt-get update
sudo apt-get install samba

Kerberos es tracta d'un protocol que serveix per a autenticar dos dispositius que es connecten entre si. No significa que vagi a autoritzar-los, sinó autenticar-los. La seva funció és identificar cada usuari mitjançant xifratge.

In [ ]:
sudo apt-get update
sudo apt-get install krb5-config

Kerberos us demana introduir 3 noms:

*   Reino de Kerberos: **EXEMPLE.COM**  # Tot en majúscules
*   Servidors de Kerberos que utilitzarem: **HOSTNAME.exemple.com**   # Tot en minúscules. Cal afegir el vostre hostname davant del nom del domini. 
*   Servidor administratiu de Kerberos que utilitzarem: **HOSTNAME.exemple.com**



Per exemple, si el meu **HOSTNAME** és "**servnil**":

In [ ]:
Reino de Kerberos: DAM.COM
Servidors de Kerberos que utilitzarem: **servnil.dam.com**
Servidor administratiu de Kerberos que utilitzarem: **servnil.dam.com**

Si us oblideu del nom que heu posat, es troba en el fitxer de configuració de kerberos, a la ruta **/etc/krb5.conf**

Necessitem winbind per unir manualment una instància de Linux amb un domini de Microsoft Active Directory

In [ ]:
sudo apt-get install winbind

Winbind us indica que vol canviar el vostre fitxer de configuració de SAMBA (smb.conf) per un altre nou. Accepteu-ho.

Si no el teniu instal·lat, també poseu el smbclient (el client de samba)

In [ ]:
sudo apt-get install smbclient

A mesura que la vostra màquina s'iniciï, haurà de conèixer l'assignació d'alguns noms d'amfitrió a adreces IP abans que es pugui fer referència al DNS. Aquesta assignació es manté al fitxer /etc/hosts. En absència d'un servidor de noms, qualsevol programa de xarxa del vostre sistema consulta aquest fitxer per determinar l'adreça IP que correspon a un nom d'amfitrió.

In [ ]:
nano /etc/hosts

In [ ]:
IPAddress       Hostname    		   Alias
127.0.0.1			  localhost	 	       
127.0.1.1			  servnil


Heu d'afegir una nova línia en aquest fitxers /etc/hosts, després de la resta de línies, amb la **IP de la vostra màquina** servidor, el **nom del servidor de Kerberos**, i un **alias que li vulgueu donar**. Per exemple:

In [ ]:
192.168.55.102		  servnil.dam.com	 	       servnil

Feu còpia de seguretat de "smb.conf"

In [ ]:
cd /etc/samba
mv smb.conf smb.conf.backup

Cal còpia de seguretat ja que farem una configuració de Samba, però no modificant directament el smb.conf, sinó utilitzant altres eines externes.

Per a que les eines externes modifiquin el smb.conf, cal NO TENIR CAP FITXER SMB.CONF, per això fem còpia de seguretat

\

Comencem configuració de samba amb comanda:

In [ ]:
samba-tool domain provision

Deixar tots els valors per defecte, ja que ha detectat els valors del Realm que heu posat anteriorment. Si no us surt **[NOMREALM.COM]** desprès de **Realm**, proveu de donar enter sense escriure res i us dóna error, vol dir que heu configurat malament el fitxer de hosts.

*   Realm[EXAMPLE.COM]: 
*   Domain[EXAMPLE]
*   Server role[dc]: L'opció per defecte és un domain controller "dc", així que enter i endavant
*   DNS backend (SAMBA_INTERNAL, BIND9_FLATFILE, BIND9_DLZ, NONE [SAMBA_INTERNAL]: Agafar el DNS intern de SAMBA, el que està per defecte, així que enter i endavant
*   DNS forwarder IP address (write 'none' to disable forwarding) [127.0.0.53]:  **8.8.8.8**  # En aquest cas sí poseu un DNS real, per exemple, el servidor de DNS de Google, que és 8.8.8.8
*  Password: Per exemple, **123456A!** 


Deixeu tots els camps per defecte. Us hauria de sortir el nom de domini que heu posat anteriorment, DC (voleu que el servidor funcioni com a controlador de domini), i els DNS per defecte.

\

Només cal emplenar l'última opció, on cal posar una contrasenya per l'administrador del domini. Recordeu-la per més endavant!

Desprès del pas anterior, hem de moure la configuració de kerberos (krb5.conf) al directori **/etc/** :

In [ ]:
# ls -l /etc/ | grep krb  
cp /var/lib/samba/private/krb5.conf /etc/

Cal desactivar els 4 serveis següents abans de continuar (serveis estàndard de SAMBA):

In [ ]:
systemctl disable --now smbd 
systemctl disable --now nmbd 
systemctl disable --now winbind 
systemctl disable --now systemd-resolved.service 

In [ ]:
systemctl unmask samba-ad-dc.service # Servei del directori actiu. Si un servei està "masked" s'evita que s'executi de forma accidental.

In [ ]:
systemctl enable --now samba-ad-dc.service

Comprovar que estem simulant un Active Directory de Windows amb el samba:

In [ ]:
samba-tool domain level show

Cal esborrar el link simbòlic següent, ja que hem de crear el nostre "**resolv.conf**" propi. Podeu fer una còpia de seguretat del fitxer original si voleu. 

In [ ]:
rm /etc/resolv.conf
nano /etc/resolv.conf

El fitxer "**resolv.conf**" conté els servidors DNS que volem utilitzar. Per aquesta pràctica, cal que hi poseu:

In [ ]:
nameserver DNS_SERVER_IP

Per exemple:

In [ ]:
nameserver 192.168.55.102

D'aquesta manera podrem trobar els serveis de Directory Actiu del servidor a través del DNS.

# Solució a diversos problemes (saltar aquesta secció si tot funciona OK)

## Re-configurar Kerberos

Si us heu equivocat amb el nom del Realm (Reino) podeu tornar a modificar la configuració de Kerberos.

Les preguntes que fa el Kerberos durant la instal·lació s'utilitzen per configurar els fitxers **/etc/krb5.conf** i **/etc/krb5kdc/kdc.conf**. 

\

*   **/etc/krb5.conf** és utilitzat per les biblioteques kerberos 5
*   **/etc/krb5kdc/kdc.conf** configura el KDC. Si necessiteu ajustar els ajustaments del Centre de distribució de claus (KDC), editeu el fitxer i reinicieu el daemon krb5-kdc.

\

Si cal reconfigurar Kerberos des de zero, potser per canviar el nom del regne, pots fer-ho escrivint la comanda:

In [ ]:
sudo dpkg-reconfigure krb5-config

## Saber quins paquets (programes) tinc instal·lats en Ubuntu Server

Simplement utilitzeu la comanda "**apt list**", la qual imprimeix tots els paquets instal·lats, seguit de "**| grep NOM_PROGRAMA**" per filtrar. Per exemple:

In [ ]:
apt list | grep krb   # imprimeix tots els mòduls instal·lats que continguin el text "krb"

## Posar una IP estàtica al Ubuntu Server

Per evitar que el DHCP ens canvii la IP del servidor i deixi de funcionar la connexió:

Agafeu una IP que no s'estigui utilitzant i que estigui dins de la vostra xarxa, i editeu el fitxer de netplan se la següent forma:

In [ ]:
/etc/netplan/00-installer-config.yaml

Per defecte teniu:

In [ ]:
network:
  ethernets:
    enp0s3:
      dhcp4: true
  version: 2

Per exemple, si teniu una xarxa "**192.168.50.0/24**", de la qual al porta d'enllaç (gateway) és "**192.168.50.1**", i escolliu una ip com "**192.168.50.144**":

Cal que ho modifiqueu de la forma següent:

In [ ]:
network:
  ethernets:
    enp0s3:
      dhcp4: no
      addresses: [192.168.50.144/24]
      nameservers:
         addresses: [8.8.8.8]
  version: 2

Quan editeu fitxers Yaml, assegureu-vos que seguiu els estàndards de sagnat de codi YAML. Si la sintaxi no és correcta, els canvis no s'aplicaran.

Una vegada fet, deseu el fitxer i apliqueu els canvis executant la següent comanda:

In [ ]:
sudo netplan apply

## Habilitar connexió al nostre DNS al firewall des de Windows

Activar programa "Cliente telnet" des de "Activar o desactivar características de Windows"

In [ ]:
telnet IP_SERVIDOR 53

Si es connecta amb telnet, hauríeu de veure un cursor parpellejant a la finestra del cmd. Si no, dirà "la connexió ha fallat".

Si la prova telnet al port 53 es connecta, llavors nslookup hauria de funcionar. Executeu-lo la comanda:

In [ ]:
nslookup - IP_SERVIDOR

# Client Windows

## PAS 1: Tenir connexió al Servidor de Directori Actiu des del client Windows

Dins de les propietats de Windows, canviar les propietats de **l'adaptador en xarxa que estigui connectat a la mateixa xarxa que el vostre servidor de Directori Actiu** (és a dir, la xarxa que us permet connectar-vos entre màquines virtuals). 

\

En l'apartat IPv4 de la configuració de l'adaptador, cal posar com a DNS la IP de la màquina del Ubuntu Server. Trobareu aquest apartat en "**Configuración de red avanzada / Cambiar opciones del adaptador** si feu anar una nova versió de Windows.

El DNS preferit ha de ser el nostre servidor de Directori Actiu, és a dir, poseu la IP DEL SERVIDOR. Exemple:

In [ ]:
DNS Preferido: 192.168.55.102

És molt recomanable seleccionar l'opció "**comprovar configuración al salir**", ja que d'aquesta manera es comprovarà si el servidor de DNS funciona correctament o no. 

## PAS 2: Unir el client al Domini

Aneu a **Configuración / Sistema / Acerca de**. Llavors, a la part dreta, premeu la opció **Cambiar el nombre de este equipo (avanzado)**.

En la finestra veureu que hi ha un botó que serveix per canviar "**El nombre de este equipo o el dominio o grupo de trabajo**".

Obreu la finestra, i a nosatres ens interessa canviar el **Domini**. El Grup de Treball (Workgroup) el podeu deixar igual. 

### Entenent perquè cal modificar el DNS preferit i altres configuracions anteriors

Ara us podeu donar compte de que formeu part, per exemple, del domini "**dam.com**". 

Necessitem que el DNS preferit sigui el que tenim a la nostra màquina virtual de Ubuntu Server, ja que el domini "**dam.com**" podria existir realment, i per tant si el nostre servidor DNS és un d'extern, tindriem la IP del servidor que guarda la pàgina web "**dam.com**" en comptes del nostre controlador de Dominis!

# Servidor Ubuntu: creació d'usuaris pel nostre domini

Amb l'ajuda de la interfície **samba-tool** podeu gestionar directament els usuaris i grups de domini, la política de grups de domini, els llocs de domini, els serveis DNS, la replicació de domini i altres funcions de domini crítiques.

Nosatres primer provarem algunes funcions més bàsiques:

## Creació d'usuaris

Amb aquesta comanda podeu crear un usuari. La mateixa comanda us demanarà que poseu una contrasenya per l'usuari:

In [ ]:
samba-tool user create NOM_USUARI

In [ ]:
samba-tool user add NOM_USUARI # comanda alternativa

## Llistar tots els usuaris del Domini

In [ ]:
samba-tool user list

## Esborrar usuari del domini

In [ ]:
samba-tool user delete NOM_USUARI

## Canviar contrasenya d'un usuari del Domini

In [ ]:
samba-tool user setpassword NOM_USUARI

## Habilitar o Deshabilitar un usuari del Domini

In [ ]:
samba-tool user disable NOM_USUARI
samba-tool user enable NOM_USUARI

# Grups d'usuaris en un Domini